OV-DINO
=====

**OV-DINO: Unified Open-Vocabulary Detection with Language-Aware Selective Fusion**

* Paper: https://arxiv.org/abs/2407.07844

![OV-DINO vs Previous Models](../assets/ovdino_vs_previous.jpg)

![OV-DINO's LASF](../assets/ovdino_lasf_overview.jpg)

![OV-DINO Overview](../assets/ovdino_overview.jpg)

## Installation

```bash
conda create -n ovdino python=3.10 -y
conda activate ovdino

# Install PyTorch for CUDA 11.6 from the official channels
conda install -y pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 \
                pytorch-cuda=11.6 -c pytorch -c nvidia

# Optional: install GCC 9 for compatibility when compiling detectron2
conda install -y gcc=9 gxx=9 -c conda-forge

# install cudatoolkit-11.6
conda install -y cudatoolkit-dev=11.6 -c conda-forge

# set CUDA_HOME env-var:
export CUDA_HOME=$CONDA_PREFIX  # since cudatoolkit-dev installs CUDA here
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib:$LD_LIBRARY_PATH

# Verify nvcc version
nvcc -V
# It should report "Cuda compilation tools, release 11.6".
```
 
```bash
git clone https://github.com/wanghao9610/OV-DINO.git OVDINO_repo
cd OVDINO_repo
export root_dir=$(realpath ./)
cd $root_dir/ovdino

# build and install detectron2
python -m pip install -e detectron2-717ab9

# install OV-DINO itself
pip install -e .
```